In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [4]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='23cd7dab-170d-4ff3-b91b-79da18b71258'),
              AIMessage(content="\n\nGreen is a wonderful colour! It's so calming and natural. What is it about green that you like the most? Is it the way it makes you think of nature and the outdoors, or is there something else that draws you to it?", additional_kwargs={'usage': {'prompt_tokens': 15, 'completion_tokens': 50, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 65}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 15, 'completion_tokens': 50, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 65}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick-17b-instruct-v1:

In [8]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='5f58d5dd-a6dc-4c8b-9416-b4a5843c2458'),
              AIMessage(content="\n\nI'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you have! How can I assist you today?", additional_kwargs={'usage': {'prompt_tokens': 16, 'completion_tokens': 41, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 57}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 16, 'completion_tokens': 41, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 57}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick-17b-instruct-v1:0'},

## Read state

In [9]:
from langchain.agents import create_agent
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    #"amazon.nova-lite-v1:0",
    llm,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [10]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is purple")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content='My favourite colour is purple', additional_kwargs={}, response_metadata={}, id='1312ece8-9bef-46af-a66e-ff8806387027'),
              AIMessage(content="\n\nPurple is a beautiful and rich colour. It's often associated with creativity, luxury, and wisdom. What is it about purple that you like the most? Is it a particular shade, like lavender or plum, or is it the way it makes you feel?", additional_kwargs={'usage': {'prompt_tokens': 15, 'completion_tokens': 53, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 68}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 15, 'completion_tokens': 53, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 68}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content='My favourite colour is purple', additional_kwargs={}, response_metadata={}, id='1312ece8-9bef-46af-a66e-ff8806387027'),
              AIMessage(content="\n\nPurple is a beautiful and rich colour. It's often associated with creativity, luxury, and wisdom. What is it about purple that you like the most? Is it a particular shade, like lavender or plum, or is it the way it makes you feel?", additional_kwargs={'usage': {'prompt_tokens': 15, 'completion_tokens': 53, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 68}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 15, 'completion_tokens': 53, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 68}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick

In [ ]:
from langchain_aws import ChatBedrock  # ChatBedrock normal
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# Estos modelos SÍ funcionan con ChatBedrock:
working_models = [
    "anthropic.claude-sonnet-4-20250514-v3:0",  # Claude
    "amazon.nova-pro-v1:0",                       # Nova
    "mistral.mistral-large-2407-v1:0",           # Mistral
]

llm = ChatBedrock(
    model_id="mistral.mistral-large-2407-v1:0",  # Usa uno de estos
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.1,
        "max_tokens": 1024,
    }
)

agent = create_agent(
    llm,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [ ]:
# model_id = "cohere.command-r-plus-v1:0"
model_id = "us.meta.llama4-scout-17b-instruct-v1:0"

# Usar ChatBedrockConverse en lugar de ChatBedrock
llm = ChatBedrockConverse(
    model=model_id,
    region_name="us-east-1",
    temperature=0.1,
    max_tokens=1024,
)

agent = create_agent(
    llm,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

response = agent.invoke(
    {"messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)


pprint(response)